# 5. BERT 기반 단일 문장 분류 모델 학습

## 강의 소개

3강에서 배운 BERT를 가지고 자연어 처리 Task를 해결해 봅니다. 🧐

단일 문장 분류 모델은 주어진 문장에 대하여 특정 라벨을 예측하는 것입니다.

활용 분야로는 리뷰의 긍정/부정 등의 감성 분석, 뉴스의 카테고리 분류, 비속어 판단 모델 등이 있습니다.

<br>

## Reference

- classfication
    - [BERT Text Classification Using Pytorch](https://towardsdatascience.com/bert-text-classification-using-pytorch-723dfb8b6b5b)
    - [Sentiment Analysis with BERT](https://curiousily.com/posts/sentiment-analysis-with-bert-and-hugging-face-using-pytorch-and-python/)
    - [네이버 영화리뷰 감정분석](https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP)
    - [Sequence Classification using Pytorch Lightning with BERT](https://medium.com/@knswamy/sequence-classification-using-pytorch-lightning-with-bert-on-imbd-data-5e9f48baa638)
    - [BERT Fine-Tuning Tutorial with PyTorch](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)

<br>

## 5.1 KLUE 데이터셋 소개

### 5.1.1 한국어 자연어 이해 벤치마크 (Korean Language Understanding Evaluation, KLUE)

- 살면서 직면하게 되는 거의 모든 자연어 task의 유형

<br>

- 문장 분류
  - 단일 문장 분류 task (5강)
- 관계 추출
  - 단일 문장 분류 task (5강)
- 문장 유사도
  - 문장 임베딩 벡터의 유사도 (e.g., [CLS])
- 자연어 추론
  - 두 문장 관계 분류 task (6강)
- 개체명 인식
  - 문장 토큰 분류 task (7강)
- 품사 태깅
  - 문장 토큰 분류 task (7강)
- 질의 응답
  - 문장 토큰 분류 task (7강)
- 목적형 대화
  - 김성동님, DST 강의
- 의존 구문 분석

<br>

### 5.1.2 의존 구문 분석

- 단어들 사이의 관계를 분석하는 task

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1TnPIBBKub5n2Ajbxabzdsdzvstv73ZCx' width=800/>

<br>

#### 5.1.2.1 특징

- 지배소: 의미의 중심이 되는 요소
- 의존소: 지배소가 갖는 의미를 보완해주는 요소 (수식)
- 어순과 생략이 자유로운 한국어와 같은 언어에서 주로 연구된다.

<br>

#### 5.1.2.2 분류 규칙

- 지배소는 후위언어이다.
  - 즉, 지배소는 항상 의존소보다 뒤에 위치한다.
  - 각 의존소의 지배소는 하나이다.
  - 교차 의존 구조는 없다.

<br>

#### 5.1.2.3 분류 방법

- Sequence labeling 방식으로 처리 단계를 나눈다.
- 앞 어절에 의존소가 없고 다음 어절이 지배소인 어절을 삭제하며 의존 관계를 만든다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1TvM1m45aswOc_oD57mIDQVM_Qql1bzIU' width=800/>

<br>

#### 5.1.2.4 사용 분야

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1TvuJXti5n01OfuuozAGi6LjaDV7BF7Gz' width=500/>

- 복잡한 **자연어** 형태를 그래프로 **구조화**해서 표현 가능
- 각 **대상(Entity)**에 대한 정보 추출 가능
  - 구름그림 -> 새털구름을 그린 것 -> 내가 그린 것
  - "나"는 "구름그림"을 그렸다.
  - "구름 그림"은 "새털구름"을 그린 것이다.

<br>

## 5.2 단일 문장 분류 task 소개

### 5.2.1 문장 분류 task

- 주어진 문장이 어떤 종류의 범주에 속하는 지를 구분하는 task

<br>

#### 5.2.1.1 감정분석(Sentiment Analysis)

- 문장의 긍정 또는 부정 및 중립 등 성향을 분류하는 프로세스
- 문장을 작성한 사람의 느낌, 감정 등을 분석할 수 있기 때문에 기업에서 모니터링, 고객지원, 또는 댓글에 대한 필터링 등을 자동화하는 작업에 주로 사용
- 활용 방안
  - 혐오 발언 분류: 댓글, 게임 대화 등 혐오 발언을 분류하여 조치를 취하는 용도로 활용
  - 기업 모니터링: 소셜, 리뷰 등 데이터에 대해 기업 이미지, 브랜드 선호도, 제품평가 등 긍정 또는 부정적인 요인을 분석

<br>

#### 5.2.1.2 주제 라벨링(Topic Labeling)

- 문장의 내용을 이해하고 적절한에 범주를 분류하는 프로세스
- 주제별로 뉴스 기사를 구성하는 등 데이터 구조화와 구성에 용이
- 활용 방안
  - 대용량 문서 분류: 대용량 문서 분류
  - VoC(Voice of Customer): 고객의 피드백을 제품 가격, 개선점, 디자인 등 적절한 주제로 분류하여 데이터를 구조화


<br>

#### 5.2.1.3 언어감지 (Language Detection)

- 문장이 어떤 나라 언어인지를 분류하는 프로세스
- 주로 번역기에서 정확한 번역을 위해 입력 문장이 어떤 나라의 언어인지 타켓팅 하는 작업이 가능
- 활용 방안
  - 번역기: 번역할 문장에 대해 적절한 언어를 감지함
  - 데이터 필터링: 타겟 언어 이외 데이터는 필터링

<br>

#### 5.2.1.4 의도 분류 (Intent Classification)

- 문장이 가진 의도를 분류하는 프로세스
- 입력 문장이 질문, 불만, 명령 등 다양한 의도를 가질 수 있기 때문에 적절한 피드백을 줄 수 있는 곳으로 라우팅 작업이 가능
- 활용 방안
  - 챗봇: 문장의 의도인 질문, 명령, 거절 등을 분석하고 적절한 답변을 주기 위해 활용

<br>

### 5.2.2 문장 분류를 위한 데이터

#### 5.2.2.1 Kor_hate

- 혐오 문장 분류
  - 혐오 표현에 대한 데이터
  - 특정 개인 또는 집단에 대한 공격적 문장
  - 무례, 공격적이거나 비꼬는 문장
  - 부정적이지 않은 문장

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Ty0ocF7W1xkE4_uySTe30wYlvtSQKjI2' width=800/>

<br>

#### 5.2.2.2 Kor_sarcasm

- 비꼬는 문장 분류
  - 비꼬지 않은 표현의 문장
  - 비꼬는 표현의 문장

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1U1DzL13EkbQWx2TYZ4JQTidP-ekBTFlX' width=800/>

<br>

#### 5.2.2.3 Kor_sae

- 질문의 유형 분류
  - 예/아니오로 답변 가능한 질문
  - 대안 선택을 묻는 질문
  - Wh- 질문 (who, what, where, when, why, how)
  - 금지 명령
  - 요구 명령
  - 강한 요구 명령

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1U9fdO9Epf9B7NHb9ds5gzlKQifb89hEM' width=800/>

<br>

#### 5.2.2.4 Kor_3i4k

- 문장의 종류 분류
  - 단어 또는 문장 조각
  - 평서문
  - 질문
  - 명령문
  - 수사적 질문
  - 수사적 명령문
  - 억양에 의존하는 의도

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1UAwPkWM_7P3bhsMYZBvgABaOZRa_wk5V' width=600/>

<br>

## 5.3 단일 문장 분류모델 학습

- BERT를 활용해 단일 문장 분류 모델 학습

<br>

### 5.3.1 모델 구조도

- BERT의 [CLS] token의 vector를 classification하는 Dense layer 사용

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1UB5UE0XF5xE4DfZyP1SHWPwheGJkRlEY' width=800/>

<br>

### 5.3.2 주요 매개변수

- `input_ids`
  - sequence token을 입력
- `attention_mask`
  - [0,1]로 구성된 마스크이며 패딩 토큰을 구분
- `token_type_ids`
  - [0,1]로 구성되었으며 입력의 첫 문장과 두번째 문장 구분
- `position_ids`
  - 각 입력 시퀀스의 임베딩 인덱스
- `inputs_embeds`
  - input_ids대신 직접 임베딩 표현을 할당
- `labels`
  - loss 계산을 위한 레이블
- `Next_sentence_label`
  - 다음 문장 예측 loss 계산을 위한 레이블

<br>

### 5.3.3 학습 과정

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1UBSZQ-turd4VX-oZdIwUoINc1xg9PqJM' width=800/>